在训练参数化机器学习模型时，权重衰减（weight decay）是最⼴泛使⽤的正则化的技术之⼀，它通常也被称为L2正则化。这项技术通过函数与零的距离来衡量函数的复杂度，因为在所有函数f中，函数f = 0（所有输⼊都得到值0）  

In [5]:
!pip install d2l
import torch
from torch import nn
from d2l import torch as d2l

In [6]:
n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5
true_w, true_b = torch.ones((num_inputs, 1)) * 0.01, 0.05
train_data = d2l.synthetic_data(true_w, true_b, n_train)
train_iter = d2l.load_array(train_data, batch_size)
test_data = d2l.synthetic_data(true_w, true_b, n_test)
test_iter = d2l.load_array(test_data, batch_size, is_train=False)

In [9]:
def train_concise(wd):
    net = nn.Sequential(nn.Linear(num_inputs,1))
    # 使用正太分布初始化权重
    for param in net.parameters():
        param.data.normal_()
    # 直接使用损失值
    loss = nn.MSELoss(reduction='none')
    num_epochs,lr = 100, 0.003
    
    trainer = torch.optim.SGD([
        {"params":net[0].weight,'weight_decay':wd},
        {"params":net[0].bias}],lr = lr)
    animator = d2l.Animator(xlabel='epochs',ylabel='loss',yscale='log',
                           xlim=[5,num_epochs],legend=['train','test'])
    for epoch in range(num_epochs):
        for X,y in train_iter:
            trainer.zero_grad()
            l = loss(net(X),y)
            l.mean().backward()
            trainer.step()
        if (epoch + 1) % 5 == 0:
            animator.add(epoch + 1,
                        (d2l.evaluate_loss(net, train_iter, loss),
                         d2l.evaluate_loss(net, test_iter, loss)))
        print('w的L2范数: ', net[0].weight.norm().item())

In [10]:
train_concise(0)

In [11]:
train_concise(3)

暂退法在前向传播过程中，计算每⼀内部层的同时注⼊噪声，这已经成为训练神经⽹络的常⽤技术。这种⽅法之所以被称为暂退法，因为我们从表⾯上看是在训练过程中丢弃（dropout）⼀些神经元。在整个训练过程的每⼀次迭代中，标准暂退法包括在计算下⼀层之前将当前层中的⼀些节点置零。

In [21]:
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784,256),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(256,256),
                    nn.ReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(256,10))
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights)
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss(reduction='none')
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,trainer)

解决（或⾄少减轻）梯度消失和梯度爆炸的⼀种⽅法是进⾏参数初始化，优化期间的注意和适当的正则化也可以进⼀步提⾼稳定性。
- 默认初始化
- Xavier初始化 Xavier初始化从均值为零，⽅差$\sigma^2=\frac{2}{n_{in}+n_{out}} $ 的⾼斯分布中采样权重

分布偏移
- 协变量偏移 虽然输⼊的分布可能随时间⽽改变，但标签函数（即条件分布$P(y|x) $没有改变
- 标签偏移 标签边缘概率$P(x|y) $可以改变，但是类别条件分布P(x|y)在不同的领域之间保持不变
- 概念偏移 标签的定义发⽣变化